In [ ]:
import subprocess
from subprocess import PIPE
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import List

In [ ]:
# Change working directory to the PathTracer
#os.chdir("C:/Users/mathias.gammelmark/Documents/repo/LightSourceImportanceSampling/PathTracer")
os.chdir("../PathTracer")

In [ ]:
def run(args:List[str]):
    process = subprocess.Popen(["../bin/Release-windows-x86_64/PathTracer.exe"] + args, stdout=subprocess.PIPE)
    while process.poll() is None:
        output = process.stdout.readline().decode("utf-8").strip()
        if (output is not ''):
            print(output)

def arg_num_samples(n:int):
    return ["-n", str(n)]

def arg_load_obj(filepath:str):
    return ["-obj", filepath]

def arg_scene(files:List[str]):
    res = []
    for file in files:
        res = res + arg_load_obj(file)
    return res

def arg_cam_pos(x:float, y:float, z:float):
    return ["-pos", str(x), str(y), str(z)]

def arg_cam_rot(x:float, y:float, z:float):
    return ["-rot", str(x), str(y), str(z)]

def arg_output(filepath:str):
    return ["-out", filepath]

def arg_outdir(filepath:str):
    if (filepath.endswith("/")):
        return ["-outdir", filepath]
    else:
        return ["-outdir", filepath + "/"]



In [ ]:
figure_dpi = 100

def load_profile_from_csv(filepath:str):
    df = pd.read_csv(filepath, header=None).T
    df.columns = df.iloc[0]
    df.drop(0,inplace=True)
    print("Loaded Profile: {}".format(filepath))
    return df

def load_image_from_csv(filepath:str):
    data = pd.read_csv(filepath).to_numpy()
    #print(np.shape(data))
    img = np.zeros((512,512,3), dtype=np.float)

    for y in range(0,511):
        for x in range(0, 511):
            pixel = data[(511-y)*512 + x][:]
            img[y,x] = pixel

    print("Loaded Image: {}".format(filepath))
    return color_correct(img)

def exposure(img, exposure):
    # reinhard tone mapping
    return img / (img+exposure)

def gamma(img, gamma):
    return np.power(img, 1.0 / gamma)

def color_correct(img):
    return gamma(exposure(img, 1.0), 2.2)

def difference(ImgA, ImgB):
    diff = np.abs(ImgA - ImgB)
    return diff[:,:,0] + diff[:,:,1] + diff[:,:,2]

def show_img(img):
    plt.figure(dpi=figure_dpi)
    plt.imshow(img)
    plt.show()

def show_spectral(img):
    plt.figure(dpi=figure_dpi)
    plt.imshow(img, cmap="coolwarm")
    plt.clim(0, 1)
    plt.colorbar()
    plt.show()

def show_diff_image(imgA, imgB):
    diff = difference(imgA, imgB)
    #diff = np.power(diff,2)
    #diff = exposure(diff, 1.0)
    show_spectral(diff)


In [ ]:
def mse(imageA, imageB):
	# the 'Mean Squared Error' between the two images is the
	# sum of the squared difference between the two images;
	# NOTE: the two images must have the same dimension
	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	err /= float(imageA.shape[0] * imageA.shape[1])
	
	# return the MSE, the lower the error, the more "similar"
	# the two images are
	return err


In [ ]:
model_cornell = "../Assets/Models/CornellBox.obj"
model_helix = "../Assets/Models/Helix.obj"
model_buddha = "../Assets/Models/Buddha.obj"

folder_results = "../Test/Results/"
folder_ref = "../Test/Ref/"

cornell_scene_args = arg_scene([model_cornell, model_helix])

In [ ]:
reference_count = 10000
def run_cornell_reference():
    args = []
    args += cornell_scene_args
    args += arg_outdir(folder_ref)
    args += arg_num_samples(reference_count)
    args += ["-method", "naive"]
    args += ["-name", "cornell_ref"]
    run(args)

# Takes a very long time to compute
# run_cornell_reference()

In [ ]:
runs = 10
scene_args = cornell_scene_args

method = "lighttree"
attenuation_type = "mindist" 

def run_statistics_test(name:str):
    models = cornell_scene_args
    static_args = (
        scene_args + 
        arg_outdir(folder_results) + 
        ["-method", method] + 
        ["-atten", attenuation_type])

    for samples in [1,2,4,8,16,32,64,128,256]:
        args = static_args
        args += arg_num_samples(samples)
        args += ["-name", name + "_spp_" + str(samples)]
        run(args)

def load_statistics_test_data(name:str):
    images = []
    profiles = []
    for samples in [1,2,4,8,16,32,64,128,256]:
        filename = folder_results + name + "_spp_" + str(samples)
        images.append(load_image_from_csv(filename + ".csv"))
        profiles.append(load_profile_from_csv(filename + "_profile.csv"))

    img_ref = load_image_from_csv(folder_ref + "cornell_ref.csv")

    result = pd.concat(profiles)
    error = np.array([mse(img, img_ref) for img in images])
    result['mse'] = error

    return result

#run_statistics_test("cornell_lighttree")
df_lighttree = load_statistics_test_data("cornell_lighttree")

In [ ]:
method = "energy"
#run_statistics_test("cornell_energy")
df_energy = load_statistics_test_data("cornell_energy")

In [ ]:
method = "naive"
#run_statistics_test("cornell_naive")
df_naive = load_statistics_test_data("cornell_naive")

In [ ]:
plt.figure()

df = pd.concat([df_energy,df_lighttree, df_naive])
df['num_samples'] = df['num_samples'].astype(int)
#df['time_render'] = df['time_render'].astype(float)

df = df.groupby(['num_samples','sampling']).mean().unstack()
df = df.plot(logx=True, logy=True)
#print(df)
#df.plot(x='time_render',logy=True)
#df_lighttree.plot(x='num_samples', y=['mse'], logx=True)
#df_energy.plot(x='num_samples', y=['mse'], logx=True)
plt.show()

In [ ]:
print("Running Pathtracer: Lighttree")
#run(cornell_scene_args + arg_num_samples(100) + ["-atten", "mindist", "-method", "lighttree", "-name", "lighttree"] + arg_outdir("../Test/Results/"))

In [ ]:
print("Running Pathtracer: Energy")
#run(cornell_scene_args + arg_num_samples(100) + ["-atten", "mindist", "-method", "energy", "-name", "Energy"] + arg_outdir("../Test/Results/"))

In [ ]:
print("Running Pathtracer: Naive")
#run(cornell_scene_args + arg_num_samples(100) + ["-atten", "mindist", "-method", "naive", "-name", "Naive"] + arg_outdir("../Test/Results/"))

In [ ]:
print("Loading Images")
res_ref = load_image_from_csv("../Test/Ref/cornell_ref.csv")
resA = load_image_from_csv("../Test/Results/Lighttree.csv")
resB = load_image_from_csv("../Test/Results/Energy.csv")
resC = load_image_from_csv("../Test/Results/Naive.csv")

print("Lighttree MSE: {}".format(mse(resA, res_ref)))
print("Energy    MSE: {}".format(mse(resB, res_ref)))
print("Naive     MSE: {}".format(mse(resC, res_ref)))

show_diff_image(resA,res_ref)
show_diff_image(resB,res_ref)
show_diff_image(resC,res_ref)

In [ ]:
frames = []
frames.append(load_profile_from_csv("../Test/Results/Lighttree_profile.csv"))
frames.append(load_profile_from_csv("../Test/Results/Energy_profile.csv"))
frames.append(load_profile_from_csv("../Test/Results/Naive_profile.csv"))

result = pd.concat(frames)
result.head()

In [ ]:
show_img((resA))
show_img((resB))
show_img((resC))
show_img((res_ref))